## Модель для ФМ 1

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from datetime import datetime
from sklearn.model_selection import GridSearchCV

In [11]:
base_path = r'C:\Users\User\Desktop\Проекты\Хакатон_норникель\dataset\\'
file_name = r'df_hack_final.csv'
test_file = r'test.csv'

df = pd.read_csv(base_path+file_name)
test = pd.read_csv(base_path+test_file)

#предобоработка даннны
df['MEAS_DT'] = pd.to_datetime(df['MEAS_DT'])
df['Ni_rec'] = df['Ni_rec'].replace({1: None})

In [12]:
def making_features_1fm(df, avg_std_start, shift, col, cols):
    model_1fm = df[cols]
    model_1fm = model_1fm.reset_index(drop=True)
    model_1fm['avg_std'] = model_1fm[col].expanding().std()
    model_1fm.at[0, 'avg_std'] = avg_std_start
    model_1fm.dropna(inplace=True)
    model_1fm['target'] = model_1fm[col].shift(shift)
    model_1fm.dropna(inplace=True)
    model_1fm['target'] = ((model_1fm['target'] > (model_1fm['avg_std']+model_1fm[col])) |
                        (model_1fm['target'] < (model_1fm[col]-model_1fm['avg_std'])))
    model_1fm['target'] = model_1fm['target'].astype(int)

    return model_1fm

In [13]:
date_start = datetime(2024, 10, 6, 0, 0, 0)
date_end = datetime(2024, 11, 11, 0, 0, 0)
df = df.query('MEAS_DT>@date_start & MEAS_DT<@date_end')

cols_train = ['Ore_mass','Mass_1','Dens_1','Cu_1.1C','Ni_1.1C', 'Cu_oreth', 'Ni_oreth', 'Cu_resth', 'Ni_resth']
cols_test = ['Ore_mass','Mass_1','Dens_1','Cu_1.2C','Ni_1.2C', 'Cu_oreth', 'Ni_oreth', 'Cu_resth', 'Ni_resth']
train_cu = making_features_1fm(df, 0.1, -8, 'Cu_1.1C', cols_train)
test_cu = making_features_1fm(df, 0.1, -8, 'Cu_1.2C', cols_test)
train_ni = making_features_1fm(df, 0.1, -8, 'Ni_1.1C', cols_train)
test_ni = making_features_1fm(df, 0.07, -8, 'Ni_1.2C', cols_test)

train_cu = train_cu.rename(columns={'Cu_1.1C':'concentration_cu', 'Cu_1.2C':'concentration_cu', 'Ni_1.1C':'concentration_ni', 'Ni_1.2C':'concentration_ni'})
test_cu = test_cu.rename(columns={'Cu_1.1C':'concentration_cu', 'Cu_1.2C':'concentration_cu', 'Ni_1.1C':'concentration_ni', 'Ni_1.2C':'concentration_ni'})
train_ni = train_ni.rename(columns={'Cu_1.1C':'concentration_cu', 'Cu_1.2C':'concentration_cu', 'Ni_1.1C':'concentration_ni', 'Ni_1.2C':'concentration_ni'})
test_ni = test_ni.rename(columns={'Cu_1.1C':'concentration_cu', 'Cu_1.2C':'concentration_cu', 'Ni_1.1C':'concentration_ni', 'Ni_1.2C':'concentration_ni'})

In [18]:
features = train_cu.drop("target", axis=1)
target = train_cu['target']


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, stratify=target,random_state=179)


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [2, 3, 5],
}

model = RandomForestClassifier(random_state=179)

# Best parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
# grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1_macro')
# grid_search.fit(X_train, y_train)
# best_params = grid_search.best_params_
# print("Best parameters:", best_params)

tunned_model = RandomForestClassifier(max_depth=None, min_samples_leaf= 2, min_samples_split=5, n_estimators=300)
tunned_model.fit(X_train, y_train)

print("Accuracy:", tunned_model.score(X_test, y_test))

valid_features = test_cu.drop("target", axis=1)
valid_target = test_cu['target']




Accuracy: 0.7274305555555556


In [16]:
predictions = tunned_model.predict(valid_features)
valid_accuracy = accuracy_score(valid_target, predictions)
valid_f1 = f1_score(valid_target, predictions)

print("Validation Accuracy:", valid_accuracy)
print("F1 Score:", valid_f1)

Validation Accuracy: 0.5898026315789474
F1 Score: 0.4937068615509541
